In [ ]:
!pip install transformers
!pip install -U torchtext==0.10.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.7 MB/s 
     |████████████████████████████████| 101 kB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 30.6 MB/s 
     |████████████████████████████████| 596 kB 58.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 7.3 MB/s 
     |████████████████████████████████| 831.4 MB 2.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.0
    Uninstalling torchtext-0.13.0:
      Successfully 

## Preparing Data

In [ ]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
len(tokenizer.vocab)

30522

In [ ]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


In [ ]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [ ]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [ ]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
from torchtext.legacy import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

We load the data and create the validation splits.

In [ ]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 58.6MB/s]


In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [ ]:
print(vars(train_data.examples[6]))

{'text': [1045, 2288, 2023, 3185, 2138, 1045, 2499, 2012, 1037, 3185, 3573, 2061, 1045, 2288, 2489, 12635, 2015, 1012, 2009, 2234, 1999, 1010, 1998, 1996, 3104, 2081, 2009, 2298, 10303, 1012, 2980, 14556, 1010, 4755, 1037, 5195, 1010, 10303, 1010, 1045, 1005, 2222, 4638, 2009, 2041, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2821, 2158, 1010, 2919, 2693, 1012, 2023, 2001, 2061, 9202, 1010, 1045, 2985, 2431, 1996, 3185, 3666, 1999, 3435, 1011, 2830, 2000, 2131, 2000, 1996, 16371, 25469, 1010, 2029, 2001, 10124, 1012, 1045, 2228, 2672, 2093, 5019, 1997, 7704, 16371, 25469, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 18178, 2229, 2100, 7982, 1010, 10231, 7685, 4808, 1010, 3532, 21917, 1997, 3494, 1012, 1045, 2514, 2919, 5128, 2023, 3185, 2091, 1010, 2138, 1045, 2113, 2009, 2001, 2081, 2006, 1037, 10036, 5166, 1010, 2021, 2061, 2001, 1000, 26706, 1000, 1998, 2009, 2150, 1037, 8754, 4438, 1998, 1037, 6329, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1016, 1013

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['i', 'got', 'this', 'movie', 'because', 'i', 'worked', 'at', 'a', 'movie', 'store', 'so', 'i', 'got', 'free', 'rental', '##s', '.', 'it', 'came', 'in', ',', 'and', 'the', 'cover', 'made', 'it', 'look', 'alright', '.', 'hot', 'chick', ',', 'carrying', 'a', 'weapon', ',', 'alright', ',', 'i', "'", 'll', 'check', 'it', 'out', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'oh', 'man', ',', 'bad', 'move', '.', 'this', 'was', 'so', 'horrible', ',', 'i', 'spent', 'half', 'the', 'movie', 'watching', 'in', 'fast', '-', 'forward', 'to', 'get', 'to', 'the', 'nu', '##dity', ',', 'which', 'was', 'minimal', '.', 'i', 'think', 'maybe', 'three', 'scenes', 'of', 'partial', 'nu', '##dity', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'che', '##es', '##y', 'dialogue', ',', 'crap', '##py', 'violence', ',', 'poor', 'excuses', 'of', 'characters', '.', 'i', 'feel', 'bad', 'putting', 'this', 'movie', 'down', ',', 'because', 'i', 'know', 'it', 'was', 'made', 'on', 'a', 'cheap', 'budget', ',', 'but', 'so', 

In [ ]:
LABEL.build_vocab(train_data)

In [ ]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Build the Model

Next, load the pre-trained model.

In [ ]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

Next, we create an instance of our model using standard hyperparameters.

In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [ ]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


## Train the Model

As is standard, we define our optimizer and criterion (loss function).

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

Next, we'll define functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 7m 7s
	Train Loss: 0.481 | Train Acc: 75.98%
	 Val. Loss: 0.270 |  Val. Acc: 89.25%
Epoch: 02 | Epoch Time: 7m 7s
	Train Loss: 0.277 | Train Acc: 88.76%
	 Val. Loss: 0.228 |  Val. Acc: 90.80%
Epoch: 03 | Epoch Time: 7m 7s
	Train Loss: 0.233 | Train Acc: 90.83%
	 Val. Loss: 0.241 |  Val. Acc: 90.42%
Epoch: 04 | Epoch Time: 7m 7s
	Train Loss: 0.209 | Train Acc: 91.83%
	 Val. Loss: 0.216 |  Val. Acc: 91.66%
Epoch: 05 | Epoch Time: 7m 7s
	Train Loss: 0.184 | Train Acc: 92.77%
	 Val. Loss: 0.214 |  Val. Acc: 91.68%


We'll load up the parameters that gave us the best validation loss and try these on the test set - which gives us our best results so far!

In [ ]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.209 | Test Acc: 91.60%


## Prediction

In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, tokenizer, "This film is terrible")

0.04567323252558708

In [ ]:
predict_sentiment(model, tokenizer, "This film is great")

0.9445724487304688

In [ ]:
predict_sentiment(model, tokenizer, "I love this video! It's really funny.")

0.9603914022445679

In [ ]:
predict_sentiment(model, tokenizer, "Well if someone like you can be called a youtuber, I guess anyone can be a youtuber.")

0.4438072144985199

In [ ]:
import pandas as pd

df = pd.read_csv('/content/poli_comments.csv')

In [ ]:
print(df.head())

                                             Comment Wing
0  I think history is completely different than w...    C
1  The wooden bird was created for those sailing ...    C
2  The tall vertical tail meant that this bird wa...    C
3  I think the wooden bird flies because it was m...    C
4  Also the aileron was ahead of the rudder. They...    C


In [ ]:
result = []
i = 0
for i in range(len(df)):
  temp = []
  temp.append(df.iloc[i,0])
  temp.append(predict_sentiment(model, tokenizer, df.iloc[i, 0]))
  temp.append(df.iloc[i,1])
  result.append(temp)
  i += 1
  print(i)

In [ ]:
import csv
head = ['Comment', 'Sentiment', 'Wing']
with open('final_result.csv', 'w+') as f:
    write = csv.writer(f) 
    write.writerow(head) 
    write.writerows(result)

In [ ]:
for row in result:
  if row[1] > 0.8:
    row.append('positive')
  elif (row[1] >= 0.4 and row[1] <= 0.6):
    row.append('neutral')
  elif row[1] < 0.2:
    row.append('negative')
  elif row[1] >= 0.2 and row[1] < 0.4:
    row.append('neutrally negative')
  elif row[1] > 0.6 and row[1] <= 0.8:
    row.append('neutrally positive')

In [ ]:
import csv
head = ['Comment', 'Score', 'Wing', 'Sentiment']
with open('final_result.csv', 'w+') as f:
    write = csv.writer(f) 
    write.writerow(head) 
    write.writerows(result)